In [17]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
from langchain_community.tools.tavily_search import TavilySearchResults
import os
from langgraph.prebuilt import create_react_agent
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import uvicorn


In [18]:
load_dotenv()

os.environ["GROQ_API_KEY"] = "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"
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

MODEL_NAMES = [
    "llama3-70b-8192",
    "mixtral-8x7b-32768"
]

In [36]:
tool_tavily = TavilySearchResults(max_results=2)
tools = [tool_tavily]


In [33]:
toola = TavilySearchResults()
print(toola.invoke({"query": "cloud trends 2025"}))

[{'url': 'https://www.softwareone.com/en-us/blog/articles/2025/01/09/4-big-picture-trends-influencing-cloud-in-2025', 'content': 'Cloud Services The demand for data centre power is set to grow 160% by 2030 as the use of AI, alongside cloud computing capabilities, increases and as a result, emissions are set to surge. The combination of edge computing – the practice of processing data near the edge of the network where it is generated - and 5G will have a significant impact on the cloud landscape. Quantum computing is one of the biggest trends set to disrupt the cloud services industry in 2025. By helping clients to migrate and modernize their workloads and applications – and in parallel, to navigate and optimize the resulting software and cloud changes – SoftwareOne unlocks the value of technology.'}, {'url': 'https://www.ascendcloudsolutions.com/the-cloud-in-2025-a-meta-analysis-of-trends-and-predictions', 'content': "Cloud migration Private cloud migration Cloud solutions for the hea

In [41]:
llm = ChatGroq(
        model="llama-3.3-70b-versatile",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2
    )

In [21]:
system_prompt = "you are a researcher"
messages = "what are the trends of AI in 2025"

In [22]:
agent = create_react_agent(llm, tools=tools, prompt=system_prompt)

In [23]:
agent.invoke({"messages": [("user", messages)]})

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n\t"tool\\_call": {\n\t\t"id": "pending",\n\t\t"type": "function",\n\t\t"function": {\n\t\t\t"name": "tavily\\_search\\_results\\_json"\n\t\t},\n\t\t"parameters": {\n\t\t\t"query": "trends of AI in 2025"\n\t\t}\n\t}\n}\n</tool-use>'}}

In [34]:
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a researcher. Use tools to find up-to-date info."),
    MessagesPlaceholder(variable_name="messages"),
    HumanMessage(content="Always use the correct JSON format for tool calls!"),
])

# alternatively, this can be passed as a function, e.g.
# def prompt(state: AgentState):
#     return (
#         [SystemMessage(content="You are a helpful assistant. Respond only in Spanish.")] +
#         state["messages"] +
#         [HumanMessage(content="Also say 'Pandamonium!' after the answer.")]
#     )


langgraph_agent_executor = create_react_agent(model=llm, tools=tools, prompt=prompt)


from langchain_core.messages import HumanMessage

messages = langgraph_agent_executor.invoke({
    "messages": [HumanMessage(content="What are the trends in cloud in 2025")]
})
print(
    {
        "input": "What are the trends in AI in 2025",
        "output": messages["messages"][-1].content,
    }
)

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': 'I apologize for the mistake. Here is the correct JSON format for the tool call:\n\n<tool-use>\n{\n\t"tool\\_call": {\n\t\t"id": "call\\_qbva",\n\t\t"type": "function",\n\t\t"function": {\n\t\t\t"name": "tavily\\_search\\_results\\_json"\n\t\t},\n\t\t"parameters": {\n\t\t\t"query": "trends in cloud in 2025"\n\t\t}\n\t}\n}\n</tool-use>\n\nThe trends in cloud in 2025 include the increasing demand for data center power, the growth of edge computing and 5G, the impact of quantum computing, and the adoption of multi-cloud strategies, AI and machine learning, enhanced security and compliance solutions, and serverless computing. Cloud security threats and solutions will continue to develop, and cloud computing will help various industries become more productiv

In [28]:
messages

{'messages': [HumanMessage(content='What are the trends in AI in 2025', additional_kwargs={}, response_metadata={}, id='46a615cf-e786-4673-bda1-ccabf1a7d43b'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fxph', 'function': {'arguments': '{"query":"trends in AI 2025"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 1233, 'total_tokens': 1337, 'completion_time': 0.160970842, 'prompt_time': 0.11482305, 'queue_time': 0.01747910500000001, 'total_time': 0.275793892}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9bacbcd5-6d67-4a0d-ac38-2b2d04ac9852-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'trends in AI 2025'}, 'id': 'call_fxph', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1233, 'output_tokens': 104, 'total_tokens': 1337}),
  ToolMessage(content=

In [37]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([("system", "You are a helpful bot named Fred."),("placeholder", "{messages}"),("user", "Remember, always be polite!"),])
graph = create_react_agent(model=llm, tools=tools, prompt=prompt)
inputs = {"messages": [("user", "What's your name? And what's the weather in SF?")]}
for s in graph.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What's your name? And what's the weather in SF?
================================== Ai Message ==================================

Of course! I'm Fred, a helpful bot at your service. I'm afraid I can't provide information about the weather in SF right now, as I don't have real-time data access or the ability to call external tools. However, I'll do my best to assist you with any other questions or tasks you have.


In [38]:
inputs

{'messages': [('user', "What's your name? And what's the weather in SF?")]}

In [39]:
import datetime

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

today = datetime.datetime.today().strftime("%D")
prompt = ChatPromptTemplate(
    [
        ("system", f"You are a helpful assistant. The date today is {today}."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

# specifying tool_choice will force the model to call this tool.
llm_with_tools = llm.bind_tools(tools)

llm_chain = prompt | llm_with_tools


@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = tool_tavily.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


tool_chain.invoke("who won the last womens singles wimbledon")

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n\t"tool\\_call": {\n\t\t"id": "pending",\n\t\t"type": "function",\n\t\t"function": {\n\t\t\t"name": "tavily\\_search\\_results\\_json"\n\t\t},\n\t\t"parameters": {\n\t\t\t"query": "last women\'s singles wimbledon winner"\n\t\t}\n\t}\n}\n</tool-use>'}}

In [44]:
import os
from dotenv import load_dotenv
from langgraph.graph import StateGraph

from langchain_community.tools import TavilySearchResults
from typing import TypedDict, Dict, List
import time
from langchain_core.output_parsers import StrOutputParser

# Load environment variables
load_dotenv()


# Define structured state
class ArticleState(TypedDict):
    subject: str
    content_details: str
    revised: str  # Holds the latest version of the article
    critique: str
    references: List[str]
    search_queries: List[str]
    external_information: List[str]
    iteration_count: int  # Tracks the number of iterations


# Initialize Tavily tool
tavily_tool = TavilySearchResults(max_results=3)

# Initialize GPT-4o as the LLM
#llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Add a rate limiter decorator
def rate_limited(max_tokens_per_minute=6000, estimated_tokens_per_call=1500):
    interval = 60 / (max_tokens_per_minute / estimated_tokens_per_call)
    def decorator(func):
        def wrapper(*args, **kwargs):
            time.sleep(interval)
            return func(*args, **kwargs)
        return wrapper
    return decorator

# Add truncation function for long content
def truncate_content(text, max_tokens=2000):
    return ' '.join(text.split()[:max_tokens])
# 1. Generate an Initial Draft
def generate_draft(state: ArticleState) -> ArticleState:
    """Generates an initial draft of the article based on the subject and content details."""
    prompt = f"""
    Write an article on the subject: "{state['subject']}". 
    Details to cover: {state['content_details']}.

    The article should include an introduction, a well-structured body, and a conclusion.
    """
    response = llm.invoke(prompt)

    return {
        **state,
        "revised": response.content or "Initial draft content placeholder.",
        "iteration_count": 0,  # Initialize iteration counter
    }


# 2. Revise the Draft with Citations
def revise_draft(state: ArticleState) -> ArticleState:
    """Refines the article, integrating references and improving clarity."""
    prompt = f"""
    Improve the following article by making it clearer, more concise, and more accurate.
    Ensure numerical citations in [#] format and add a references section.

    Subject: {state['subject']}
    Content Details: {state['content_details']}

    Current Draft:
    {state['revised']}
    """
    response = llm.invoke(prompt)

    return {**state, "revised": response.content or state["revised"]}


# 3. Critique and Reflection
def critique_article(state: ArticleState) -> ArticleState:
    """Provides a critique of the current version of the article and suggests improvements."""
    prompt = f"""
    Critique the following article. Identify weaknesses, missing information, and unnecessary content.
    Provide three key areas for improvement.

    Subject: {state['subject']}
    Content Details: {state['content_details']}

    Article:
    {state['revised']}
    """
    response = llm.invoke(prompt)

    return {**state, "critique": response.content or "No critique available."}


# 4. Generate Research Queries
def generate_search_queries(state: ArticleState) -> ArticleState:
    """Generate 3 focused queries instead of 5"""
    prompt = f"""
    Generate 3 focused research queries to improve: "{state['subject']}".
    Focus on finding technical documentation and recent trends.
    """
    response = llm.invoke(prompt)
    queries = response.content.split("\n")[:3]  # Take only first 3
    return {**state, "search_queries": queries}


# 5. Fetch External Information
def fetch_external_information(state: ArticleState) -> ArticleState:
    """Fetch and filter external info"""
    external_info = []
    for query in state["search_queries"][:3]:  # Process only first 3 queries
        if query.strip():
            print(f"🔍 Searching for: {query}")
            try:
                results = tavily_tool.invoke(query)
                # Extract just the relevant snippets
                if results and 'results' in results:
                    external_info.extend([
                        f"Source: {res['title']}\nSnippet: {res['content'][:200]}..."
                        for res in results['results'][:2]  # Take top 2 results
                    ])
            except Exception as e:
                print(f"⚠️ Error searching {query}: {e}")
    return {**state, "external_information": external_info}

# 6. Iterative Refinement
@rate_limited()
def iterative_refinement(state: ArticleState) -> ArticleState:
    """Refines the article based on critique and external information."""
    # Truncate inputs to stay within token limits
    truncated_info = truncate_content(str(state["external_information"]))
    truncated_critique = truncate_content(state["critique"])
    truncated_draft = truncate_content(state["revised"])

    # Create the prompt
    prompt = f"""
    Update and improve the article on "{state['subject']}" using the feedback and research below.
    Ensure all claims are well-supported and properly cited.

    External Information:
    {truncated_info}

    Critique:
    {truncated_critique}

    Current Draft:
    {truncated_draft}
    """
    
    # Invoke the LLM
    response = llm.invoke(prompt)

    # Return the updated state
    return {
        **state,  # Preserve all existing state fields
        "revised": response.content,  # Update the revised content
        "iteration_count": state["iteration_count"] + 1,  # Increment counter
    }


# 7. Final Step
def final_step(state: ArticleState) -> ArticleState:
    """Finalizes the article after three iterations."""
    return {
        **state,
        "revised": state["revised"]
        + f"\n\nFinalized after 3 iterations.\n\nArticle on: {state['subject']}.",
    }


# 8. Conditional Check for Looping
def should_continue(state: ArticleState) -> str:
    """Determines whether to refine again or finish."""
    return "critique_article" if state["iteration_count"] < 1 else "final_step"


# Create LangGraph State Machine
workflow = StateGraph(ArticleState)

# Add Nodes
workflow.add_node("generate_draft", generate_draft)
workflow.add_node("revise_draft", revise_draft)
workflow.add_node("critique_article", critique_article)
workflow.add_node("generate_search_queries", generate_search_queries)
workflow.add_node("fetch_external_information", fetch_external_information)
workflow.add_node("iterative_refinement", iterative_refinement)
workflow.add_node("final_step", final_step)

# Define Execution Flow
workflow.set_entry_point("generate_draft")
workflow.add_edge("generate_draft", "revise_draft")
workflow.add_edge("revise_draft", "critique_article")
workflow.add_edge("critique_article", "generate_search_queries")
workflow.add_edge("generate_search_queries", "fetch_external_information")
workflow.add_edge("fetch_external_information", "iterative_refinement")

# Add Conditional Edge for Looping
workflow.add_conditional_edges(
    "iterative_refinement",
    should_continue,
    {"critique_article": "critique_article", "final_step": "final_step"},
)

# Set Final Step
workflow.set_finish_point("final_step")

# Compile the workflow
article_workflow = workflow.compile()

# Input Message with Subject and Content Details
input_message = {
    "subject": "Building a LangGraph Workflow with Tavily and GPT-4o",
    "content_details": "The article should cover state management in LangGraph, the role of Tavily in web search, how GPT-4o enhances summarization, and a step-by-step implementation guide."
}

# Run the Workflow
final_result = article_workflow.invoke(input_message)

# Display the Final Refined Article
print("\n🔹 **Final Article:**")
print(final_result["revised"])
# print(article_workflow.get_graph().draw_mermaid())

🔍 Searching for: Here are three focused research queries to improve "Building a LangGraph Workflow with Tavily and GPT-4o":
🔍 Searching for: 1. **Technical Integration Query**:

🔹 **Final Article:**
**Building a LangGraph Workflow with Tavily and GPT-4o: A Comprehensive Guide**

### Introduction

The field of natural language processing (NLP) has witnessed significant advancements in recent years, transforming the way we interact with and process language-based data. LangGraph, a powerful tool for managing and analyzing complex language structures, can be combined with Tavily, a cutting-edge web search engine, and GPT-4o, an advanced language model, to increase the potential for efficient and accurate language processing workflows. This article provides a comprehensive overview of building a LangGraph workflow with Tavily and GPT-4o, focusing on state management in LangGraph, the role of Tavily in web search, and how GPT-4o enhances summarization.

### State Management in LangGraph

La

In [45]:
final_result

{'subject': 'Building a LangGraph Workflow with Tavily and GPT-4o',
 'content_details': 'The article should cover state management in LangGraph, the role of Tavily in web search, how GPT-4o enhances summarization, and a step-by-step implementation guide.',
 'revised': '**Building a LangGraph Workflow with Tavily and GPT-4o: A Comprehensive Guide**\n====================================================================\n\n### Introduction\n\nThe field of natural language processing (NLP) has witnessed significant advancements in recent years, transforming the way we interact with and process language-based data. LangGraph, a powerful tool for managing and analyzing complex language structures, can be combined with Tavily, a cutting-edge web search engine, and GPT-4o, an advanced language model, to increase the potential for efficient and accurate language processing workflows. This article provides a comprehensive overview of building a LangGraph workflow with Tavily and GPT-4o, focusing 